In [1]:
# Install necessary dependencies
%pip install langchain openai neo4j-driver python-dotenv neo4j
%pip install langchain_community
%pip install langchain_openai
%pip install pypdf
%pip install python-dotenv
%pip install langchain-experimental

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import os
# import warnings
# from typing import List, Optional, Any, Dict

# from dotenv import load_dotenv

# from langchain_core.documents import Document
# from pydantic import BaseModel, Field
# from langchain_community.document_loaders import PyPDFLoader, TextLoader
# from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# # Suppress specific warnings from langchain_experimental
# warnings.filterwarnings(
#     "ignore",
#     message="Importing LLMGraphTransformer from langchain_experimental.graph_transformers is deprecated",
#     category=UserWarning
# )

# # --- Configuration ---
# # Note: 'gpt-4o-2024-11-20' might not be a valid model name.
# # Using 'gpt-4o' which is the latest generally available version as of mid-2024.
# # Ensure your API key has access to this model.
# LLM_MODEL_NAME = "gpt-4o"
# EMBEDDING_MODEL_NAME = "text-embedding-3-small" # Or another OpenAI embedding model
# # Define the types of entities and relationships you want to extract (optional but recommended)
# # If None, the LLM will try to infer them.
# ALLOWED_NODES = ["Person", "Organization", "Location", "Concept", "Event", "Document"]
# ALLOWED_RELATIONSHIPS = ["CONNECTS", "MENTIONS", "LOCATED_IN", "PART_OF", "RELATED_TO", "AUTHOR_OF", "REFERENCES"]

# # --- Pydantic Models for Structured Output (Used internally by LLMGraphTransformer) ---
# # These help structure the LLM's output but you primarily interact with the GraphDocument object

# class Node(BaseModel):
#     """Represents a node in the graph."""
#     id: str = Field(description="Unique identifier for the node (e.g., entity name)")
#     type: str = Field(description="Type of the node (e.g., Person, Organization)")
#     properties: Optional[Dict[str, Any]] = Field(
#         description="Additional properties associated with the node", default_factory=dict
#     )

# class Relationship(BaseModel):
#     """Represents a relationship between two nodes in the graph."""
#     source: Node = Field(description="The source node of the relationship")
#     target: Node = Field(description="The target node of the relationship")
#     type: str = Field(description="The type of the relationship (e.g., CONNECTS, MENTIONS)")
#     properties: Optional[Dict[str, Any]] = Field(
#         description="Additional properties associated with the relationship", default_factory=dict
#     )

# class KnowledgeGraph(BaseModel):
#     """Represents the structured knowledge graph extracted from a text chunk."""
#     nodes: List[Node] = Field(description="List of nodes in the graph")
#     relationships: List[Relationship] = Field(description="List of relationships in the graph")

# # --- Core Functions ---

# def check_openai_connection(api_key: str, model_name: str) -> bool:
#     """Checks if the OpenAI API key and model are accessible."""
#     print(f"Attempting to connect to OpenAI with model: {model_name}...")
#     try:
#         llm = ChatOpenAI(openai_api_key=api_key, model=model_name, temperature=0)
#         # Simple test invocation
#         llm.invoke("Hello!")
#         print("OpenAI connection successful.")
#         return True
#     except Exception as e:
#         print(f"Error connecting to OpenAI or accessing model '{model_name}': {e}")
#         print("Please check your API key, model access permissions, and network connection.")
#         return False

# def process_file(file_path: str, llm: ChatOpenAI, embeddings_model: OpenAIEmbeddings) -> List[Document]:
#     """
#     Loads, splits, embeds, and extracts graph data from a text or PDF file.

#     Args:
#         file_path: Path to the input file (.txt or .pdf).
#         llm: Initialized ChatOpenAI model for extraction.
#         embeddings_model: Initialized OpenAIEmbeddings model.

#     Returns:
#         A list of LangChain GraphDocument objects, each representing the
#         extracted graph structure from a chunk of the original document.
#         Note: Embeddings are created internally if needed by the transformer
#               but the primary output here is the structured graph data.
#     """
#     print(f"\nProcessing file: {file_path}...")
#     file_extension = os.path.splitext(file_path)[1].lower()

#     if file_extension == ".txt":
#         loader = TextLoader(file_path, encoding='utf-8')
#     elif file_extension == ".pdf":
#         loader = PyPDFLoader(file_path)
#     else:
#         print(f"Unsupported file type: {file_extension}. Skipping.")
#         return []

#     try:
#         documents = loader.load() # Loads and potentially splits based on loader (PDF often splits by page)
#         print(f"Loaded {len(documents)} initial document sections.")

#         # Using LLMGraphTransformer to handle chunking and extraction
#         # It uses the LLM to identify nodes and relationships based on the text.
#         graph_transformer = LLMGraphTransformer(
#             llm=llm,
#             allowed_nodes=ALLOWED_NODES,               # Optional: Guide the LLM
#             allowed_relationships=ALLOWED_RELATIONSHIPS, # Optional: Guide the LLM
#             prompt=None # Use default prompt or provide a custom one
#         )

#         print("Extracting graph structure from documents...")
#         # This is the core step: converts text documents into graph documents
#         graph_documents = graph_transformer.convert_to_graph_documents(documents)
#         print(f"Extracted {len(graph_documents)} graph documents.")

#         # Note: Embeddings are not explicitly added to the output here,
#         # as the primary goal was entity/relationship extraction for Memgraph.
#         # The transformer might use embeddings internally if configured,
#         # but the returned GraphDocument focuses on nodes and relationships.
#         # If you need chunk text + embedding + graph, you'd need a more manual loop.

#         return graph_documents

#     except Exception as e:
#         print(f"Error processing file {file_path}: {e}")
#         return []

# # --- Main Execution ---

# if __name__ == "__main__":
#     load_dotenv()
#     api_key = os.getenv("OPENAI_API_KEY")

#     if not api_key:
#         print("Error: OPENAI_API_KEY not found in environment variables.")
#         exit(1)

#     # 1. Check OpenAI Connection First
#     if not check_openai_connection(api_key, LLM_MODEL_NAME):
#         exit(1) # Stop if connection fails

#     # 2. Initialize models (only after successful connection check)
#     llm = ChatOpenAI(openai_api_key=api_key, model=LLM_MODEL_NAME, temperature=0)
#     embeddings = OpenAIEmbeddings(openai_api_key=api_key, model=EMBEDDING_MODEL_NAME)

#     # 3. Define input file path (replace with your file)
#     # input_file = "path/to/your/document.txt"
#     input_file = "/Users/ryangichuru/Downloads/AI Group Project/other/resources/Need Help in a Crisis 2025 (1).pdf" # Or your PDF file

#     if not os.path.exists(input_file):
#         print(f"Error: Input file not found at {input_file}")
#         exit(1)

#     # 4. Process the file
#     processed_graph_data = process_file(input_file, llm, embeddings)

#     # 5. Display results (example: print nodes and relationships for the first graph doc)
#     if processed_graph_data:
#         print("\n--- Example Extracted Graph Data (First Chunk) ---")
#         first_graph_doc = processed_graph_data[0]

#         print("\nNodes:")
#         for node in first_graph_doc.nodes:
#             print(f"  ID: {node.id}, Type: {node.type}, Properties: {node.properties}")

#         print("\nRelationships:")
#         for rel in first_graph_doc.relationships:
#             print(f"  Source: {rel.source.id} ({rel.source.type})")
#             print(f"  Target: {rel.target.id} ({rel.target.type})")
#             print(f"  Type: {rel.type}")
#             print(f"  Properties: {rel.properties}")
#             print("-" * 10)

#         print(f"\nSuccessfully processed file. Found {len(processed_graph_data)} graph segments.")
#         # You would typically save this structured data (e.g., to JSON)
#         # for later ingestion into Memgraph using Cypher queries.
#     else:
#         print("No graph data extracted.")

In [4]:
# /Users/ryangichuru/Downloads/AI Group Project/other/data_extractor.ipynb
import os
import warnings
from typing import List, Optional, Any, Dict
import json
from pathlib import Path
import traceback # Import traceback for better error logging

from dotenv import load_dotenv

from langchain_core.documents import Document
# Updated Pydantic imports for v2 compatibility
from pydantic import BaseModel, Field, field_validator # Changed validator to field_validator
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate

# Suppress specific warnings
warnings.filterwarnings(
    "ignore",
    message="Importing LLMGraphTransformer from langchain_experimental.graph_transformers is deprecated",
    category=UserWarning
)
warnings.filterwarnings(
    "ignore",
    message=".*LangChain uses pydantic v2 internally.*",
    category=UserWarning
)
warnings.filterwarnings(
    "ignore",
    message="Pydantic V1 style `@validator` validators are deprecated.*",
    category=DeprecationWarning # Pydantic uses DeprecationWarning for this
)
warnings.filterwarnings(
    "ignore",
    message=".*you should migrate to Pydantic V2 style `@field_validator` validators.*",
    category=DeprecationWarning # Pydantic uses DeprecationWarning for this
)


# --- Configuration ---
LLM_MODEL_NAME = "gpt-4o"
EMBEDDING_MODEL_NAME = "text-embedding-3-small"
ALLOWED_NODES = ["Person", "Organization", "Location", "Concept", "Event", "Document", "Service", "Condition", "Guidance", "Resource", "Contact"]
ALLOWED_RELATIONSHIPS = ["CONNECTS", "MENTIONS", "LOCATED_IN", "PART_OF", "RELATED_TO", "AUTHOR_OF", "REFERENCES", "PROVIDES", "HELPS_WITH", "CONTACT_FOR", "OFFERS"]

# --- Custom Prompt Template String ---
# Removed the literal {} from the instructions text.
CUSTOM_PROMPT_TEMPLATE_STR = """
Extract entities and relationships from the text below based on the provided schema.

Instructions:
1. Identify entities (nodes) and relationships relevant to the allowed types.
2. For each node, provide its 'id' (unique name/identifier), 'type', and a 'properties' dictionary containing relevant attributes found in the text (e.g., for a 'Service', extract 'phone_number', 'website'; for a 'Location', extract 'address'). If no properties are found, the value for 'properties' MUST be an empty JSON dictionary.
3. For each relationship, provide its 'source' node 'id', 'target' node 'id', 'type', and a 'properties' dictionary describing the relationship (e.g., for 'CONTACT_FOR', specify the contact detail if it's part of the relationship context). If no properties are found, the value for 'properties' MUST be an empty JSON dictionary.
4. Ensure node 'id' values are consistent across the graph. If an entity is mentioned multiple times, use the same 'id'.
5. Output ONLY a valid JSON object containing two keys: 'nodes' and 'relationships'. The value for each key should be a list of the extracted nodes/relationships. Adhere strictly to the JSON format. Do not include explanations or markdown formatting.

Permitted Node Types:
{node_labels}

Permitted Relationship Types:
{rel_labels}

Text:
{input}

JSON Output:
"""

# --- Pydantic Models for Structured Output ---

class Node(BaseModel):
    """Represents a node in the graph."""
    id: str = Field(description="Unique identifier for the node (e.g., entity name or concept)")
    type: str = Field(description="Type of the node (e.g., Person, Organization, Concept)")
    properties: Optional[Dict[str, Any]] = Field(
        description="Additional properties associated with the node", default_factory=dict
    )

    @field_validator('properties', mode='before')
    @classmethod
    def set_properties_default(cls, v):
        return v or {}


class Relationship(BaseModel):
    """Represents a relationship between two nodes in the graph."""
    source: Node = Field(description="The source node of the relationship")
    target: Node = Field(description="The target node of the relationship")
    type: str = Field(description="The type of the relationship (e.g., CONNECTS, MENTIONS)")
    properties: Optional[Dict[str, Any]] = Field(
        description="Additional properties associated with the relationship", default_factory=dict
    )

    @field_validator('properties', mode='before')
    @classmethod
    def set_properties_default(cls, v):
        return v or {}


class KnowledgeGraph(BaseModel):
    """Represents the structured knowledge graph extracted from a text chunk."""
    nodes: List[Node] = Field(description="List of nodes in the graph")
    relationships: List[Relationship] = Field(description="List of relationships in the graph")


# --- Core Functions ---

def check_openai_connection(api_key: str, model_name: str) -> bool:
    """Checks if the OpenAI API key and model are accessible."""
    print(f"Attempting to connect to OpenAI with model: {model_name}...")
    try:
        llm = ChatOpenAI(openai_api_key=api_key, model=model_name, temperature=0)
        llm.invoke("Hello!")
        print("OpenAI connection successful.")
        return True
    except Exception as e:
        print(f"Error connecting to OpenAI or accessing model '{model_name}': {e}")
        print("Please check your API key, model access permissions, and network connection.")
        return False

def process_file(file_path: Path, llm: ChatOpenAI, embeddings_model: OpenAIEmbeddings) -> List[Document]:
    """
    Loads, splits, embeds, and extracts graph data from a text or PDF file.

    Args:
        file_path: Path object for the input file (.txt or .pdf).
        llm: Initialized ChatOpenAI model for extraction.
        embeddings_model: Initialized OpenAIEmbeddings model.

    Returns:
        A list of LangChain GraphDocument objects.
    """
    print(f"\nProcessing file: {file_path.name}...")
    file_extension = file_path.suffix.lower()

    if file_extension == ".txt":
        loader = TextLoader(str(file_path), encoding='utf-8')
    elif file_extension == ".pdf":
        loader = PyPDFLoader(str(file_path))
    else:
        print(f"Unsupported file type: {file_extension}. Skipping.")
        return []

    try:
        documents = loader.load()
        if not documents:
             print(f"No content loaded from {file_path.name}. Skipping.")
             return []
        print(f"Loaded {len(documents)} initial document sections from {file_path.name}.")

        # Format the prompt string dynamically
        formatted_prompt_str = CUSTOM_PROMPT_TEMPLATE_STR.format(
            node_labels=", ".join(ALLOWED_NODES),
            rel_labels=", ".join(ALLOWED_RELATIONSHIPS),
            input="{input}" # Keep placeholder for transformer
        )
        # Create the prompt template *after* formatting with node/rel labels
        dynamic_prompt = ChatPromptTemplate.from_template(formatted_prompt_str)


        # Initialize the transformer
        graph_transformer = LLMGraphTransformer(
            llm=llm,
            allowed_nodes=ALLOWED_NODES,
            allowed_relationships=ALLOWED_RELATIONSHIPS,
            prompt=dynamic_prompt, # Use the dynamically formatted prompt
            node_properties=True,
            relationship_properties=True,
            strict_mode=True
        )

        print(f"Extracting graph structure from {file_path.name} using custom prompt...")
        if documents:
            graph_documents = graph_transformer.convert_to_graph_documents(documents)
            print(f"Extracted {len(graph_documents)} graph documents from {file_path.name}.")
            # Filter out potentially empty graph documents if the LLM fails on a chunk
            valid_graph_documents = [gd for gd in graph_documents if gd.nodes or gd.relationships]
            if len(valid_graph_documents) < len(graph_documents):
                print(f"Filtered out {len(graph_documents) - len(valid_graph_documents)} empty graph document(s).")
            return valid_graph_documents
        else:
            print(f"No documents to process for {file_path.name}.")
            return []

    except Exception as e:
        print(f"Error processing file {file_path.name} ({type(e).__name__}): {e}")
        traceback.print_exc() # Print the full traceback for debugging
        return []

def graph_document_to_dict(graph_doc: Document) -> Dict:
    """Converts a GraphDocument to a JSON-serializable dictionary."""
    nodes_list = getattr(graph_doc, 'nodes', [])
    relationships_list = getattr(graph_doc, 'relationships', [])
    metadata = getattr(graph_doc, 'metadata', {})

    serializable_nodes = []
    for node in nodes_list:
        try:
            serializable_nodes.append(node.model_dump())
        except Exception as dump_error:
             print(f"Warning: Could not serialize node: {node}. Error: {dump_error}")

    serializable_relationships = []
    for rel in relationships_list:
        try:
            dumped_rel = rel.model_dump()
            if isinstance(dumped_rel.get("source"), BaseModel):
                 dumped_rel["source"] = dumped_rel["source"].model_dump()
            if isinstance(dumped_rel.get("target"), BaseModel):
                 dumped_rel["target"] = dumped_rel["target"].model_dump()
            serializable_relationships.append(dumped_rel)
        except Exception as dump_error:
            print(f"Warning: Could not serialize relationship: {rel}. Error: {dump_error}")


    return {
        "metadata": metadata,
        "nodes": serializable_nodes,
        "relationships": serializable_relationships
    }


# --- Main Execution ---

if __name__ == "__main__":
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")

    if not api_key:
        print("Error: OPENAI_API_KEY not found in environment variables.")
        exit(1)

    base_dir = Path("/Users/ryangichuru/Downloads/AI Group Project/other/resources")
    input_dir = base_dir
    output_dir = base_dir / "json_data"

    output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Input directory: {input_dir}")
    print(f"Output directory: {output_dir}")

    if not check_openai_connection(api_key, LLM_MODEL_NAME):
        exit(1)

    llm = ChatOpenAI(openai_api_key=api_key, model=LLM_MODEL_NAME, temperature=0)
    embeddings = OpenAIEmbeddings(openai_api_key=api_key, model=EMBEDDING_MODEL_NAME)

    files_to_process = list(input_dir.glob('*.txt')) + list(input_dir.glob('*.pdf'))

    if not files_to_process:
        print(f"No .txt or .pdf files found in {input_dir}")
        exit(0)

    print(f"\nFound {len(files_to_process)} files to process.")

    for file_path in files_to_process:
        processed_graph_data_list = process_file(file_path, llm, embeddings)

        if processed_graph_data_list:
            output_filename = output_dir / f"{file_path.stem}.json"
            output_data = [graph_document_to_dict(graph_doc) for graph_doc in processed_graph_data_list]
            output_data = [item for item in output_data if item.get("nodes") or item.get("relationships")]

            if not output_data:
                 print(f"No valid graph data extracted and converted for {file_path.name}. No JSON file created.")
                 continue

            try:
                with open(output_filename, 'w', encoding='utf-8') as f:
                    json.dump(output_data, f, indent=2, ensure_ascii=False)
                print(f"Successfully saved extracted data for {file_path.name} to {output_filename}")
            except Exception as e:
                print(f"Error saving JSON for {file_path.name}: {e}")
        else:
            print(f"No graph data extracted for {file_path.name} after processing. No JSON file created.")

    print("\n--- Processing Complete ---")


Input directory: /Users/ryangichuru/Downloads/AI Group Project/other/resources
Output directory: /Users/ryangichuru/Downloads/AI Group Project/other/resources/json_data
Attempting to connect to OpenAI with model: gpt-4o...
OpenAI connection successful.

Found 6 files to process.

Processing file: Selp help resources.txt...
Loaded 1 initial document sections from Selp help resources.txt.
Extracting graph structure from Selp help resources.txt using custom prompt...
Extracted 1 graph documents from Selp help resources.txt.
Successfully saved extracted data for Selp help resources.txt to /Users/ryangichuru/Downloads/AI Group Project/other/resources/json_data/Selp help resources.json

Processing file: Uni Mental health day resources.txt...
Loaded 1 initial document sections from Uni Mental health day resources.txt.
Extracting graph structure from Uni Mental health day resources.txt using custom prompt...
Extracted 1 graph documents from Uni Mental health day resources.txt.
Successfully sav

In [6]:
# --- Make sure these imports are at the top of your script/cell ---
import os
import json
from pathlib import Path
from typing import List, Dict, Any
from dotenv import load_dotenv

from langchain_community.graphs import MemgraphGraph
# Use GraphDocument for the import
from langchain_community.graphs.graph_document import GraphDocument, Node as LangChainNode, Relationship as LangChainRelationship
from langchain_core.documents import Document # Keep for source reference
# Import your Pydantic models (make sure they match the ones used in data_extractor.ipynb)
from pydantic import BaseModel, Field, field_validator

# --- Pydantic Models (Should match data_extractor.ipynb) ---
# These are used for deserializing the JSON
class Node(BaseModel):
    """Represents a node in the graph."""
    id: str = Field(description="Unique identifier for the node")
    type: str = Field(description="Type of the node")
    properties: Dict[str, Any] = Field(default_factory=dict)

    @field_validator('properties', mode='before')
    @classmethod
    def set_properties_default(cls, v):
        return v or {}

class Relationship(BaseModel):
    """Represents a relationship between two nodes."""
    source: Node = Field(description="The source node")
    target: Node = Field(description="The target node")
    type: str = Field(description="Type of the relationship")
    properties: Dict[str, Any] = Field(default_factory=dict)

    @field_validator('properties', mode='before')
    @classmethod
    def set_properties_default(cls, v):
        return v or {}

# Helper to structure the data read from JSON
class GraphChunk(BaseModel):
     metadata: Dict[str, Any] = Field(default_factory=dict)
     nodes: List[Node]
     relationships: List[Relationship]


# --- Main Import Function (Updated) ---
def import_json_to_memgraph(json_file_path: Path, graph: MemgraphGraph):
    """Reads the extracted JSON and imports it into Memgraph."""
    print(f"Reading JSON file: {json_file_path}...")
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data_from_json = json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found at {json_file_path}")
        return
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from {json_file_path}: {e}")
        return
    except Exception as e:
        print(f"An unexpected error occurred reading {json_file_path}: {e}")
        return

    if not isinstance(data_from_json, list):
        print(f"Error: Expected a list of graph chunks in {json_file_path}, but got {type(data_from_json)}")
        return

    graph_documents_to_add: List[GraphDocument] = [] # Correct type hint

    print(f"Deserializing {len(data_from_json)} graph chunks from JSON...")
    for i, chunk_data in enumerate(data_from_json):
        try:
            # Use Pydantic to parse and validate the chunk data from JSON
            graph_chunk = GraphChunk.model_validate(chunk_data)

            # Convert Pydantic Nodes/Relationships to LangChain versions
            lc_nodes = [LangChainNode(id=n.id, type=n.type, properties=n.properties) for n in graph_chunk.nodes]
            lc_relationships = [
                LangChainRelationship(
                    source=LangChainNode(id=r.source.id, type=r.source.type, properties=r.source.properties),
                    target=LangChainNode(id=r.target.id, type=r.target.type, properties=r.target.properties),
                    type=r.type,
                    properties=r.properties
                ) for r in graph_chunk.relationships
            ]

            # Create a source Document (optional but good practice for provenance)
            source_doc = Document(page_content="", metadata=graph_chunk.metadata) # Use an empty page_content

            # Create the GraphDocument object expected by add_graph_documents
            graph_doc = GraphDocument(
                nodes=lc_nodes,
                relationships=lc_relationships,
                source=source_doc # Link back to the source metadata
            )
            graph_documents_to_add.append(graph_doc)

        except Exception as validation_error: # Catch validation or other errors here
            print(f"Warning: Skipping chunk {i} due to validation/parsing error: {validation_error}")
            # Optionally log the problematic chunk_data for debugging
            # import logging
            # logging.warning(f"Problematic data in chunk {i} of {json_file_path.name}: {chunk_data}", exc_info=True)
            continue # Skip to the next chunk

    if not graph_documents_to_add:
        print("No valid graph documents to import after deserialization.")
        return

    print(f"Importing {len(graph_documents_to_add)} graph documents into Memgraph...")
    try:
        # This is the core LangChain step for importing
        graph.add_graph_documents(
            graph_documents_to_add,
            # include_source_info=True # Set to True if you want Document metadata stored as node properties
        )
        print(f"Import successful for {json_file_path.name}!")
    except Exception as e:
        print(f"Error during Memgraph import for {json_file_path.name}: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    load_dotenv()

    # Memgraph connection details from environment variables
    memgraph_uri = os.getenv("MEMGRAPH_URI", "bolt://localhost:7687")
    memgraph_username = os.getenv("MEMGRAPH_USERNAME", "")
    memgraph_password = os.getenv("MEMGRAPH_PASSWORD", "")

    # Directory where your JSON files are stored
    json_data_dir = Path("/Users/ryangichuru/Downloads/AI Group Project/other/resources/json_data")

    # --- Initialize Memgraph Connection ---
    try:
        print(f"Connecting to Memgraph at {memgraph_uri}...")
        memgraph = MemgraphGraph(
            url=memgraph_uri,
            username=memgraph_username,
            password=memgraph_password
        )
        # Optional: Test connection with a simple query
        memgraph.query("RETURN 1")
        print("Memgraph connection successful.")
    except Exception as e:
        print(f"Error connecting to Memgraph: {e}")
        exit(1)

    # --- Find and Process JSON Files ---
    json_files = list(json_data_dir.glob('*.json'))

    if not json_files:
        print(f"No JSON files found in {json_data_dir}")
        exit(0)

    print(f"\nFound {len(json_files)} JSON files to import.")

    for json_file in json_files:
        import_json_to_memgraph(json_file, memgraph)

    print("\n--- Import Process Complete ---")


Connecting to Memgraph at bolt://localhost:7687...
Memgraph connection successful.

Found 6 JSON files to import.
Reading JSON file: /Users/ryangichuru/Downloads/AI Group Project/other/resources/json_data/Need Help in a Crisis 2025 (1).json...
Deserializing 4 graph chunks from JSON...
Importing 4 graph documents into Memgraph...
Import successful for Need Help in a Crisis 2025 (1).json!
Reading JSON file: /Users/ryangichuru/Downloads/AI Group Project/other/resources/json_data/Selp help resources.json...
Deserializing 1 graph chunks from JSON...
Importing 1 graph documents into Memgraph...
Import successful for Selp help resources.json!
Reading JSON file: /Users/ryangichuru/Downloads/AI Group Project/other/resources/json_data/Councelling FAQs.json...
Deserializing 1 graph chunks from JSON...
Importing 1 graph documents into Memgraph...
Import successful for Councelling FAQs.json!
Reading JSON file: /Users/ryangichuru/Downloads/AI Group Project/other/resources/json_data/Councellors Corne